# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [ ]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [ ]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [ ]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [ ]:
your_name = "suiii" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [ ]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [ ]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
1178,23,female,40.79205,0,no,3044.463818
1295,20,male,25.74000,1,no,2063.019000
205,28,female,33.78960,1,no,4554.621960
1067,39,male,49.90635,0,no,6045.284123
523,38,female,44.14410,0,no,5667.497535


In [ ]:
dataframe.shape

(1271, 6)

Let us answer some basic questions about the dataset. 


**Q1: How many rows does the dataset have?**

In [ ]:
num_rows = 1271
print(num_rows)

1271


**Q2: How many columns doe the dataset have**

In [ ]:
num_cols = 6
print(num_cols)

6


**Q3: What are the column titles of the input variables?**

In [ ]:
dataframe.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges'], dtype='object')

In [ ]:
input_cols = ['age', 'sex', 'bmi', 'children', 'smoker', 'charges']

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [ ]:
categorical_cols = ['sex', 'smoker']

**Q5: What are the column titles of output/target variable(s)?**

In [ ]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

In [ ]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [ ]:
!pip install jovian --upgrade -q

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/02-insurance-linear-regression" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/aakashns/02-insurance-linear-regression


'https://jovian.ai/aakashns/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [ ]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.30000000e+01, 0.00000000e+00, 4.07920500e+01, 0.00000000e+00,
         0.00000000e+00, 3.04446382e+03],
        [2.00000000e+01, 1.00000000e+00, 2.57400000e+01, 1.00000000e+00,
         0.00000000e+00, 2.06301900e+03],
        [2.80000000e+01, 0.00000000e+00, 3.37896000e+01, 1.00000000e+00,
         0.00000000e+00, 4.55462196e+03],
        ...,
        [5.30000000e+01, 0.00000000e+00, 3.12390000e+01, 2.00000000e+00,
         0.00000000e+00, 1.17083190e+04],
        [2.60000000e+01, 1.00000000e+00, 3.16602000e+01, 0.00000000e+00,
         1.00000000e+00, 1.78955085e+04],
        [1.90000000e+01, 0.00000000e+00, 3.31227000e+01, 0.00000000e+00,
         1.00000000e+00, 1.83424331e+04]]),
 array([[ 3044.4638175],
        [ 2063.019    ],
        [ 4554.62196  ],
        ...,
        [11708.319    ],
        [17895.50847  ],
        [18342.433095 ]]))

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [ ]:
inputs = torch.tensor(inputs_array, dtype = torch.float32)
targets = torch.tensor(targets_array, dtype = torch.float32)

In [ ]:
  inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [ ]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [ ]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size
from torch.utils.data import random_split

train_ds, val_ds = random_split(dataset, [val_size, train_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [ ]:
batch_size = 100

In [ ]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [ ]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[4.3000e+01, 1.0000e+00, 2.3552e+01, 2.0000e+00, 1.0000e+00, 1.9706e+04],
        [5.7000e+01, 1.0000e+00, 4.7233e+01, 0.0000e+00, 0.0000e+00, 1.1532e+04],
        [4.5000e+01, 0.0000e+00, 4.1301e+01, 0.0000e+00, 0.0000e+00, 7.7155e+03],
        [1.9000e+01, 1.0000e+00, 2.8782e+01, 1.0000e+00, 0.0000e+00, 1.9291e+03],
        [4.0000e+01, 0.0000e+00, 2.9788e+01, 1.0000e+00, 0.0000e+00, 7.4310e+03],
        [4.4000e+01, 1.0000e+00, 4.6238e+01, 0.0000e+00, 0.0000e+00, 7.2961e+03],
        [1.8000e+01, 0.0000e+00, 3.6680e+01, 0.0000e+00, 0.0000e+00, 1.7033e+03],
        [4.7000e+01, 0.0000e+00, 2.8197e+01, 1.0000e+00, 0.0000e+00, 2.7548e+04],
        [4.7000e+01, 1.0000e+00, 3.3012e+01, 3.0000e+00, 1.0000e+00, 2.6161e+04],
        [2.9000e+01, 0.0000e+00, 3.2690e+01, 1.0000e+00, 1.0000e+00, 2.0063e+04],
        [5.1000e+01, 0.0000e+00, 3.0186e+01, 1.0000e+00, 0.0000e+00, 1.0354e+04],
        [2.4000e+01, 0.0000e+00, 2.4008e+01, 0.0000e+00, 1.0000e+00, 1.5300e+04],
        

Let's save our work by committing to Jovian.

In [ ]:
jovian.commit(project=project_name, environment=None)

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [ ]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [ ]:
import torch.nn.functional as F
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                 # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                        # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.smooth_l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.smooth_l1_loss(out, targets)    
        # acc = accuracy(out, targets)                  # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [ ]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2169,  0.2466,  0.3187,  0.1971, -0.0769,  0.3118]],
        requires_grad=True),
 Parameter containing:
 tensor([0.1359], requires_grad=True)]

One final commit before we train the model.

In [ ]:
jovian.commit(project=project_name, environment=None)

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [ ]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 9651.23046875}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [ ]:
epochs = 50
lr = 0.4
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 20804844.0000
Epoch [40], val_loss: 74342352.0000
Epoch [50], val_loss: 6962509.0000


In [ ]:
epochs = 1000
lr = 0.4
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 17201998.0000
Epoch [40], val_loss: 46013968.0000
Epoch [60], val_loss: 24014690.0000
Epoch [80], val_loss: 6405621.0000
Epoch [100], val_loss: 23948328.0000
Epoch [120], val_loss: 79419184.0000
Epoch [140], val_loss: 36364336.0000
Epoch [160], val_loss: 46513628.0000
Epoch [180], val_loss: 59716016.0000
Epoch [200], val_loss: 52309492.0000
Epoch [220], val_loss: 78562912.0000
Epoch [240], val_loss: 30731142.0000
Epoch [260], val_loss: 52056252.0000
Epoch [280], val_loss: 49626968.0000
Epoch [300], val_loss: 22994594.0000
Epoch [320], val_loss: 33779484.0000
Epoch [340], val_loss: 34157288.0000
Epoch [360], val_loss: 14675252.0000
Epoch [380], val_loss: 19512862.0000
Epoch [400], val_loss: 76010272.0000
Epoch [420], val_loss: 45676748.0000
Epoch [440], val_loss: 29288398.0000
Epoch [460], val_loss: 73992456.0000
Epoch [480], val_loss: 73982176.0000
Epoch [500], val_loss: 81185056.0000
Epoch [520], val_loss: 75228008.0000
Epoch [540], val_loss: 67770200.0000
Epoch 

In [ ]:
epochs = 100
lr = 0.3
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 30918458.0000
Epoch [40], val_loss: 40382956.0000
Epoch [60], val_loss: 9076915.0000
Epoch [80], val_loss: 32603406.0000
Epoch [100], val_loss: 53786904.0000


In [ ]:
epochs = 500
lr = 0.1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 12759652.0000
Epoch [40], val_loss: 4997568.5000
Epoch [60], val_loss: 7553351.5000
Epoch [80], val_loss: 863248.7500
Epoch [100], val_loss: 3180088.2500
Epoch [120], val_loss: 9718713.0000
Epoch [140], val_loss: 17178526.0000
Epoch [160], val_loss: 18997780.0000
Epoch [180], val_loss: 15283607.0000
Epoch [200], val_loss: 4461331.5000
Epoch [220], val_loss: 879852.2500
Epoch [240], val_loss: 2586388.0000
Epoch [260], val_loss: 8534391.0000
Epoch [280], val_loss: 14160957.0000
Epoch [300], val_loss: 11127086.0000
Epoch [320], val_loss: 14640233.0000
Epoch [340], val_loss: 8844682.0000
Epoch [360], val_loss: 19433672.0000
Epoch [380], val_loss: 19234940.0000
Epoch [400], val_loss: 17583722.0000
Epoch [420], val_loss: 13099555.0000
Epoch [440], val_loss: 19699854.0000
Epoch [460], val_loss: 15714909.0000
Epoch [480], val_loss: 12323011.0000
Epoch [500], val_loss: 2630360.0000


In [ ]:
epochs = 1000
lr = 0.2
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 4538938.0000
Epoch [40], val_loss: 23669406.0000
Epoch [60], val_loss: 2347005.7500
Epoch [80], val_loss: 20921152.0000
Epoch [100], val_loss: 24246338.0000
Epoch [120], val_loss: 16808696.0000
Epoch [140], val_loss: 16553868.0000
Epoch [160], val_loss: 13388700.0000
Epoch [180], val_loss: 4121723.7500
Epoch [200], val_loss: 3286051.2500
Epoch [220], val_loss: 37412732.0000
Epoch [240], val_loss: 11146775.0000
Epoch [260], val_loss: 30582626.0000
Epoch [280], val_loss: 29977984.0000
Epoch [300], val_loss: 7611324.5000
Epoch [320], val_loss: 960980.7500
Epoch [340], val_loss: 185442.2500
Epoch [360], val_loss: 6581443.0000
Epoch [380], val_loss: 36562960.0000
Epoch [400], val_loss: 26251744.0000
Epoch [420], val_loss: 22029982.0000
Epoch [440], val_loss: 1158429.0000
Epoch [460], val_loss: 40897080.0000
Epoch [480], val_loss: 36731952.0000
Epoch [500], val_loss: 23055670.0000
Epoch [520], val_loss: 4723145.5000
Epoch [540], val_loss: 24843848.0000
Epoch [560], val_

**Q12: What is the final validation loss of your model?**

In [ ]:
val_loss = 8032379.5000

Let's log the final validation loss to Jovian and commit the notebook

In [ ]:
jovian.log_metrics(val_loss=val_loss)

In [ ]:
jovian.commit(project=project_name, environment=None)

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [ ]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions =  model(inputs)          # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [ ]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([5.1000e+01, 0.0000e+00, 4.0014e+01, 1.0000e+00, 0.0000e+00, 1.0366e+04])
Target: tensor([10366.3359])
Prediction: tensor([5945379.])


In [ ]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([5.1000e+01, 1.0000e+00, 2.7156e+01, 1.0000e+00, 1.0000e+00, 2.3329e+04])
Target: tensor([23329.0215])
Prediction: tensor([13379015.])


In [ ]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([6.2000e+01, 0.0000e+00, 3.0759e+01, 0.0000e+00, 1.0000e+00, 2.9199e+04])
Target: tensor([29199.1621])
Prediction: tensor([16745461.])


Are you happy with your model's predictions? Try to improve them further.


NO :(